In [1]:
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from datetime import datetime, timedelta, timezone
import util_calendar
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']


itol = {num + 1 : chr(num + ord("A")) for num in range(0,27)}


In [2]:
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
sheets_token_loc = 'creds_google/sheets-token.pickle'
credentials_loc = 'creds_google/credentials.json'

if os.path.exists(sheets_token_loc):
    with open(sheets_token_loc, 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            credentials_loc, SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open(sheets_token_loc, 'wb') as token:
        pickle.dump(creds, token)

service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()




In [5]:
# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = '1GDXzzTD1dBnXWcpjPXqfIjpLwxHtXVQtjwfLhCaZNHA'
SHEET_NAME = 'STUDY-PLAN'
RANGE_NAME = SHEET_NAME + '!A:G'

result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                            range=RANGE_NAME).execute()

values = result.get('values', [])
values

[['Name',
  'Duration',
  'Time',
  'Date',
  'Frequency',
  'Google Calendar Invite Code',
  'Update'],
 ['IBM Data Science', '0:25', '2:00 PM', '20-Jan-2021', 'Daily'],
 ['Course on Philosophy', '0:30', '5:00 PM', '25-Jan-2021', 'Weekly']]

In [9]:

fields = values[0]
fields = [_.lower() for _ in fields]
print(f"Fields of the study plan include the following : {fields}")

# Fill this list with Events Objects
study_plans = []

for index, row in enumerate(values[1:], 2):
    
    #check row for valid values
    if "" in row:
        print("Invalid Row : There are missing values")
    elif len(row) < 5:
        print("Not enough fields present")    
    else:
        series = dict(zip(fields,row))

        # Now create Event dict
        event = { "row_index" : index,
                 "series" : series
            }
        
        # Add that event to the study_plan
        study_plans.append(event)
        
# Check to see if the event needs to be created/updated
for event in study_plans:
    
    event_code = event['series'].get('google calendar invite code')
    if event_code is not None:
        # Update the event if the Updated Column is enabled
        if event['series'].get('updated') == 'TRUE':
            pass # Update
            # finally disable the Updated Column, and add new code
        else:
            pass # Do Nothing

    else:
        # Create a new calendar event from this event
        series = event['series'] # Create an alias
        
        # Extract the Date, Time & Duration     
        start_date = datetime.strptime(series['time']+ " " + series['date'], "%I:%M %p %d-%b-%Y")
        duration = datetime.strptime(series['duration'], "%H:%M")

        # Calculate the End Date
        end_date = start_date + timedelta(minutes = duration.minute, hours = duration.hour)

        # Convert to Text ISO Format
        start_date = datetime.isoformat(start_date, timespec='milliseconds')
        end_date = datetime.isoformat(end_date, timespec='milliseconds')

        tz = 'GMT+3:00'

        # Check to see if the keywords for Frequency is valid
        # [checking function here]
        freq = series["frequency"].upper()

        event_bluprnt = {
          'summary': 'Appointment',
          'location': 'Somewhere',
          'start': {
            'dateTime': start_date,
            'timeZone': tz
          },
          'end': {
            'dateTime': end_date,
              'timeZone': tz
          },
          'recurrence': [
            f'RRULE:FREQ={freq}',
          ],
        }

        print(event_bluprnt)


        # Write the event code to the cell
        new_cal_code  = util_calendar.insert_event(event_bluprnt)
        values = [[new_cal_code]]
        body = {
            'values': values
        }
        range_name = f"{SHEET_NAME}!" + itol[6] + str(event['row_index'])
        value_input_option = 'RAW'
        result = service.spreadsheets().values().update(
            spreadsheetId=SPREADSHEET_ID, range=range_name,
            valueInputOption=value_input_option, body=body).execute()


            
            



Fields of the study plan include the following : ['name', 'duration', 'time', 'date', 'frequency', 'google calendar invite code', 'update']
{'summary': 'Appointment', 'location': 'Somewhere', 'start': {'dateTime': '2021-01-20T14:00:00.000', 'timeZone': 'GMT+3:00'}, 'end': {'dateTime': '2021-01-20T14:25:00.000', 'timeZone': 'GMT+3:00'}, 'recurrence': ['RRULE:FREQ=DAILY']}
{'summary': 'Appointment', 'location': 'Somewhere', 'start': {'dateTime': '2021-01-25T17:00:00.000', 'timeZone': 'GMT+3:00'}, 'end': {'dateTime': '2021-01-25T17:30:00.000', 'timeZone': 'GMT+3:00'}, 'recurrence': ['RRULE:FREQ=WEEKLY']}


In [ ]:
event = {
  'summary': 'Appointment',
  'location': 'Somewhere',
  'start': {
    'dateTime': '2021-01-20T10:00:00.000',
    'timeZone': 'GMT+3:00'
  },
  'end': {
    'dateTime': '2021-01-20T10:25:00.000',
      'timeZone': 'GMT+3:00'
  },
  'recurrence': [
    'RRULE:FREQ=DAILY;UNTIL=20210201T170000Z',
  ],
}



In [ ]:
# Get the RRULE date format
# date_rrtime = datetime.strftime(date, "%Y%m%dT235959Z")
# date_rrtime




